## Benjamin Wilke
## Homework 7

Imagine you are an Engineer for Amazon Alexa.  You are given the task of understanding text input and classifying it.  Create a data set of sayings that fall into 2 Categories: Getting Directions and Playing Music.  Make sure you data set has at least 100 examples total.

Example Playing Music:
"Play some Pantera on Spotify"

Example Directions:
"Where is the nearest Starbucks"

### Q1 Given the following module that outputs a vector which represents your input sentence, transform your data into numbers.

In [11]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/3")

In [1]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read().splitlines()
    file.close()
    return text

In [4]:
musicDat = load_doc("alexa_commands_music.txt")
directionsDat = load_doc("alexa_commands_directions.txt")

In [8]:
print(f'Length of Music is {len(musicDat)}, Length of Directions is {len(directionsDat)}')

Length of Music is 50, Length of Directions is 50


In [13]:
musicEmbeddings = embed(musicDat)["outputs"]
directionsEmbeddings = embed(directionsDat)["outputs"]

In [81]:
print(f'Shape of Music embeddings is {musicEmbeddings.shape}, Shape of Directions embeddings is {directionsEmbeddings.shape}')

Shape of Music embeddings is (50, 512), Shape of Directions embeddings is (50, 512)


In [59]:
musicEmbeddings[1]

<tf.Tensor: shape=(512,), dtype=float32, numpy=
array([ 5.22320718e-02,  2.61381362e-02,  8.00965130e-02, -3.76225859e-02,
        3.82037531e-03, -2.12687608e-02, -3.28792855e-02, -9.24806297e-03,
       -6.00333363e-02, -6.85106590e-02,  2.56405678e-02,  7.38508254e-02,
       -1.15180220e-02, -6.57589883e-02,  5.54771684e-02,  5.46576604e-02,
       -2.80031227e-02, -1.91100743e-02,  3.91349979e-02, -7.16580749e-02,
       -6.40604645e-02, -2.97386609e-02,  7.10452627e-03, -1.08329682e-02,
       -8.00965205e-02,  7.53745064e-02,  7.15863928e-02,  4.38992158e-02,
       -4.14860696e-02, -2.07935329e-04, -7.46235102e-02, -7.80304298e-02,
        3.23309526e-02, -1.96301639e-02, -1.67930759e-02,  1.28326667e-02,
        1.58166252e-02, -9.32152662e-03, -2.71972530e-02,  9.61224549e-03,
       -2.25116801e-03,  3.01870704e-02,  2.45128591e-02, -6.49812296e-02,
        1.42342374e-02,  4.53504510e-02,  8.68111476e-03,  1.28336893e-02,
        8.04916397e-03,  5.92718497e-02,  6.64762259

### Q2: Take your numbers from Q1 and create 2 classifiers of any type.  What is your best guess for the accuracy of your models. (Don't forget to use good data science practices!)

I'm going to run a Logistic Regression and a Random Forest on this dataset. My best guess is that these will be VERY easy to classify correctly. There are features in each that are VERY distinct - for example all of the music related commands include the word "play", while all of the direction related commands include a word like "how", "what", "which", and "are".

In [76]:
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier

In [83]:
joinedDats = np.vstack((musicEmbeddings, directionsEmbeddings))

In [19]:
joinedDats.shape

(100, 512)

In [66]:
# let's scale the data, why not?
scaler = StandardScaler()
joinedDatsScaled = scaler.fit_transform(joinedDats)

In [46]:
# let's create our labels, music = 1, directions = 0
labels = np.stack((np.ones((50,), dtype=int), np.zeros((50,), dtype=int)), axis=0).reshape(100,)

In [58]:
labels.shape

(100,)

In [60]:
labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [68]:
log_reg = LogisticRegression()
labels_pred = cross_val_score(log_reg, joinedDatsScaled, labels, cv=10)

In [75]:
print("The Logistic Regression accuracy is {}%".format(labels_pred.mean() * 100))

The Logistic Regression accuracy is 100.0%


In [78]:
rnd_clf = RandomForestClassifier()
labels_rnd_pred = cross_val_score(rnd_clf, joinedDatsScaled, labels, cv=10)

In [80]:
print("The Random Forest accuracy is {}%".format(labels_rnd_pred.mean() * 100))

The Random Forest accuracy is 100.0%
